# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

In [2]:
# import NLP libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [3]:
# download for lemmatization
#nltk.download(['punkt', 'wordnet']) 

In [ ]:
# import sklearn package
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("Disaster_messages", con=engine)

In [ ]:
df

In [ ]:
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    """
    Tokenization function to process text data
    """
    tokens = word_tokenize(text) #tokenize sentences
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = [] #save cleaned tokens
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok) # save cleaned tokens
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
# build NLP pipeline -count words, tf-idf, multiple output classifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
#split data into train set and test set
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [ ]:
# training process
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def build_report(pipeline, X_test, y_test):
    # predict X_test
    y_pred = pipeline.predict(X_test)
    
    # build classification report in columns
    performances = []
    for index, column in enumerate(y_test):
        performances.append([f1_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                             precision_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                             recall_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro')])
    
    
    # build dataframe
    performances = pd.DataFrame(performances, columns=['f1 score', 'precision', 'recall'],
                                index = y_test.columns)   
    return performances

In [ ]:
build_report(pipeline, X_test, y_test)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'clf__estimator__max_features':['sqrt', 0.5],
              'clf__estimator__n_estimators':[50, 100]}


In [ ]:
cv = GridSearchCV(estimator=pipeline, param_grid = parameters, cv = 5, n_jobs = 6)
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
build_report(cv, X_test, y_test)

In [ ]:
cv.best_params

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pickle.dump(pipeline, open('rf_model.pkl', 'wb'))

In [ ]:
pickle.dump(pipeline_improved, open('adaboost_model.pkl', 'wb'))

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.